# Crypto Forecasting 

In [12]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json

%matplotlib inline

In [13]:
# Load .env environment variables
load_dotenv("alpaca.env")

True

In [14]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [11]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2018-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2023-05-01', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["AAPL", "MSFT", "AMD"]

# Get current closing prices 
stocks_df = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
AAPL = stocks_df[stocks_df["symbol"]=="AAPL"].drop("symbol", axis=1)
MSFT = stocks_df[stocks_df["symbol"]=="MSFT"].drop("symbol", axis=1)
AMD = stocks_df[stocks_df["symbol"]=="AMD"].drop("symbol", axis=1)

# Concatenate the ticker DataFrames
stocks_df= pd.concat([AAPL, MSFT, AMD],axis=1, keys=["AAPL","MSFT", "AMD"])

# Preview DataFrame
stocks_df

AAPL                                       \
                               open     high       low   close    volume   
timestamp                                                                  
2018-05-01 04:00:00+00:00  166.4102  169.200  165.2700  169.10  53569474   
2018-05-02 04:00:00+00:00  175.2250  177.750  173.8000  176.57  66851451   
2018-05-03 04:00:00+00:00  175.8800  177.500  174.4411  176.89  34068446   
2018-05-04 04:00:00+00:00  178.2500  184.250  178.1700  183.83  56204195   
2018-05-07 04:00:00+00:00  185.1800  187.670  184.7500  185.16  42613114   
...                             ...      ...       ...     ...       ...   
2023-04-25 04:00:00+00:00  165.1900  166.305  163.7300  163.77  48849358   
2023-04-26 04:00:00+00:00  163.9000  165.280  162.8000  163.76  45524694   
2023-04-27 04:00:00+00:00  165.1900  168.560  165.1900  168.41  64891617   
2023-04-28 04:00:00+00:00  168.4900  169.850  167.8801  169.68  53888026   
2023-05-01 04:00:00+00:00  169.2800  170.450  168.6400  169.59  52520935   

                                                     MSFT                   \
                          trade_count        vwap    open     high     low   
timestamp                                                                    
2018-05-01 04:00:00+00:00      353863  169.130276   93.21   95.290   92.79   
2018-05-02 04:00:00+00:00      437602  176.126868   94.99   95.170   93.19   
2018-05-03 04:00:00+00:00      246062  176.100682   92.96   94.925   92.45   
2018-05-04 04:00:00+00:00      370771  182.554203   93.32   95.370   92.92   
2018-05-07 04:00:00+00:00      267449  186.216900   95.17   96.710   95.10   
...                               ...         ...     ...      ...     ...   
2023-04-25 04:00:00+00:00      502773  164.647271  279.51  281.600  275.37   
2023-04-26 04:00:00+00:00      494174  163.908156  296.70  299.570  292.73   
2023-04-27 04:00:00+00:00      575890  167.319460  295.97  305.200  295.25   
2023-04-28 04:00:00+00:00      513328  168.948227  304.01  308.930  303.31   
2023-05-01 04:00:00+00:00      540746  169.657850  306.97  308.600  305.15   

                           ...                                       AMD  \
                           ...    volume trade_count        vwap    open   
timestamp                  ...                                             
2018-05-01 04:00:00+00:00  ...  31409017      209726   94.199608  10.830   
2018-05-02 04:00:00+00:00  ...  27471134      188110   94.016453  11.100   
2018-05-03 04:00:00+00:00  ...  31142548      238268   93.702789  10.902   
2018-05-04 04:00:00+00:00  ...  22531383      165938   94.745049  10.880   
2018-05-07 04:00:00+00:00  ...  24262669      172194   96.187561  11.320   
...                        ...       ...         ...         ...     ...   
2023-04-25 04:00:00+00:00  ...  47004633      627981  280.958848  86.900   
2023-04-26 04:00:00+00:00  ...  64681706      869828  296.027735  86.510   
2023-04-27 04:00:00+00:00  ...  46463058      650724  302.549963  86.410   
2023-04-28 04:00:00+00:00  ...  31737282      485116  306.123532  87.020   
2023-05-01 04:00:00+00:00  ...  21321107      316910  306.614195  91.030   

                                                                         \
                             high      low  close    volume trade_count   
timestamp                                                                 
2018-05-01 04:00:00+00:00  11.150  10.7700  11.13  43441602       90980   
2018-05-02 04:00:00+00:00  11.155  10.9700  10.97  29267466       77517   
2018-05-03 04:00:00+00:00  11.060  10.8300  10.93  33310596       89301   
2018-05-04 04:00:00+00:00  11.290  10.8701  11.28  47885590      120350   
2018-05-07 04:00:00+00:00  11.680  11.3000  11.59  54003947      132612   
...                           ...      ...    ...       ...         ...   
2023-04-25 04:00:00+00:00  87.060  83.7600  83.80  52893533      342477   
2023-04-26 04:00:00+00:00  87.350  85.3200  85.94  4781519